In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-4.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
0     prop-4      285    2    3    0   16   16     1   0  16  ...  0.0    0   
1     prop-4      285    2    4    0   23   19     1   0  23  ...  0.0    0   
2     prop-4      285    4    6    0    8   13     4   0   8  ...  0.0    0   
3     prop-4      285    4    5    2   23   53     4   2  21  ...  0.0    0   
4     prop-4      285    2    2    0   11   10     1   0  11  ...  0.0    0   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
8713  prop-4      318    0    1    0    0    0     0   0   0  ...  0.0    0   
8714  prop-4      318    0    1    0    0    0     0   0   0  ...  0.0    0   
8715  prop-4      318   20    2    0   10   63   140   1   9  ...  0.8    6   
8716  prop-4      318   12    2    0    8   30    66   1   7  ...  0.0    0   
8717  prop-4      318    0    1    0    0    0     0   0   0  ...  0.0    0   

           mfa       cam  ic  cbm    amc  max_cc  avg_cc  bug  
0     0.900000  0.625000   1    1  62.50       1  0.5000    0  
1     0.933333  0.600000   1    1  34.50       1  0.5000    0  
2     0.913043  0.555556   2    2   8.75       1  0.5000    0  
3     0.960784  0.555556   1    2  70.75       1  0.5000    0  
4     0.800000  0.666667   1    1  18.50       1  0.5000    0  
...        ...       ...  ..  ...    ...     ...     ...  ...  
8713  0.000000  0.000000   0    0   0.00       0  0.0000    0  
8714  0.000000  0.000000   0    0   0.00       0  0.0000    0  
8715  0.500000  0.302632   0    0  17.20       5  1.3500    0  
8716  0.620690  0.388889   0    0  21.25       6  1.9167    0  
8717  0.000000  0.000000   0    0   0.00       0  0.0000    0  

[8718 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 285]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
0     prop-4      285    2    3    0   16   16     1   0  16  ...  0.0    0   
1     prop-4      285    2    4    0   23   19     1   0  23  ...  0.0    0   
2     prop-4      285    4    6    0    8   13     4   0   8  ...  0.0    0   
3     prop-4      285    4    5    2   23   53     4   2  21  ...  0.0    0   
4     prop-4      285    2    2    0   11   10     1   0  11  ...  0.0    0   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
1689  prop-4      285    1    1    0    1    5     0   0   1  ...  0.0    0   
1690  prop-4      285    1    1    0    0    5     0   0   0  ...  0.0    0   
1691  prop-4      285    1    1    0    2    5     0   1   1  ...  0.0    0   
1692  prop-4      285    1    1    0    1    5     0   0   1  ...  0.0    0   
1693  prop-4      285    1    1    0    0    5     0   0   0  ...  0.0    0   

           mfa       cam  ic  cbm    amc  max_cc  avg_cc  bug  
0     0.900000  0.625000   1    1  62.50       1     0.5    0  
1     0.933333  0.600000   1    1  34.50       1     0.5    0  
2     0.913043  0.555556   2    2   8.75       1     0.5    0  
3     0.960784  0.555556   1    2  70.75       1     0.5    0  
4     0.800000  0.666667   1    1  18.50       1     0.5    0  
...        ...       ...  ..  ...    ...     ...     ...  ...  
1689  0.000000  0.000000   0    0  12.00       0     0.0    0  
1690  0.000000  0.000000   0    0  12.00       0     0.0    0  
1691  0.000000  0.000000   0    0  12.00       0     0.0    0  
1692  0.000000  0.000000   0    0  12.00       0     0.0    0  
1693  0.000000  0.000000   0    0  12.00       0     0.0    0  

[1694 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
1027    5    5    0   19   30     8   2  17    3  1.000000   113  0.000000   
1278    4    1    0   28   81     4   1  27    1  1.233831  1319  0.731343   
394     2    3    0   16   19     1   0  16    2  2.000000    90  0.000000   
1494    2    5    0    1    3     1   0   1    1  2.000000     7  0.000000   
1054    3    4    0   14   12     3   0  14    1  2.000000    40  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
1150    4    4    0   24   47     4   0  24    3  1.000000   191  0.000000   
1644    3    2    0    3    7     3   0   3    1  2.000000    23  0.000000   
1125    3    3    0    8   14     1   0   8    2  0.750000    46  0.500000   
1616    3    1    0    9   19     1   0   9    3  1.300000    95  0.000000   
248     3    6    0   15   27     1   0  15    2  0.900000   128  0.200000   

      moa       mfa       cam  ic  cbm         amc  max_cc   avg_cc  
1027    0  0.942308  0.500000   4   11   21.000000       1   0.6000  
1278    0  0.000000  0.444444   0    0  312.000000     103  26.0000  
394     0  0.857143  0.625000   1    1   44.000000       1   0.5000  
1494    0  0.979592  1.000000   1    1    2.500000       1   0.5000  
1054    0  0.846154  0.533333   1    1   12.333333       1   0.6667  
...   ...       ...       ...  ..  ...         ...     ...      ...  
1150    0  0.931034  0.555556   2    2   46.000000       1   0.5000  
1644    0  0.666667  0.666667   1    2    6.666667       1   0.6667  
1125    0  0.941176  0.666667   0    0   13.666667       2   0.6667  
1616    0  0.000000  0.666667   0    0   29.000000       2   1.3333  
248     0  0.981481  0.750000   1    1   40.000000       5   1.6667  

[1355 rows x 20 columns]

In [13]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  moa  \
111   19    1    0   65   72   159  33  32   17  0.988889  398  0.400000    0   
404    2    1    0    2    6     1   0   2    2  2.000000   28  0.000000    0   
805    4    7    0   11   27     4   0  11    2  0.888889  168  0.333333    0   
454    2    3    0   13   14     1   0  13    2  2.000000   68  1.000000    0   
553    2    3    0    7   13     1   0   7    2  2.000000   51  0.000000    0   
..   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...  ...   
485    2    3    0   26   23     1   0  26    2  2.000000  123  0.000000    0   
693    5    5    0   14   24     8   1  13    3  1.000000   97  0.000000    0   
811    2    4    0   36   31     1   0  36    2  1.666667  206  0.000000    0   
981    7    7    2   18   22     9   2  16    1  0.888889   84  0.000000    0   
192    2    3    0   18   20     1   0  18    2  2.000000  106  0.000000    0   

          mfa       cam  ic  cbm         amc  max_cc  avg_cc  
111  0.000000  0.092593   0    0   19.684211       6  1.9474  
404  0.000000  0.750000   0    0   13.000000       2  1.0000  
805  0.964286  0.500000   2    2   40.250000       4  1.2500  
454  0.857143  0.625000   1    1   32.500000       1  0.5000  
553  0.857143  0.625000   1    1   24.500000       4  2.0000  
..        ...       ...  ..  ...         ...     ...     ...  
485  0.857143  0.625000   1    1   60.500000       1  0.5000  
693  0.942308  0.500000   4   11   17.800000       1  0.6000  
811  0.933333  0.600000   1    1  100.500000       1  0.5000  
981  0.821429  0.666667   3    4   10.571429       1  0.7143  
192  0.857143  0.625000   1    1   52.000000       1  0.5000  

[339 rows x 20 columns]

In [14]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [15]:
y_train

bug
1027    0
1278    2
394     0
1494    0
1054    0
...   ...
1150    1
1644    0
1125    0
1616    0
248     0

[1355 rows x 1 columns]

In [16]:
y_test

bug
111    0
404    0
805    0
454    0
553    0
..   ...
485    0
693    0
811    0
981    0
192    0

[339 rows x 1 columns]

In [17]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [18]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [19]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [20]:
y = y_test['bug'].tolist()
y

[0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv285/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
14/14 [==============================] - 1s 2ms/step - loss: 1894.5533 - mse: 1894.5533 - mae: 17.2533 - root_mean_squared_error: 43.5265 - mean_squared_logarithmic_error: 5.5370
Epoch 2/100
14/14 [==============================] - 0s 1ms/step - loss: 1602.2653 - mse: 1602.2653 - mae: 14.9650 - root_mean_squared_error: 40.0283 - mean_squared_logarithmic_error: 4.8719
Epoch 3/100
14/14 [==============================] - 0s 1ms/step - loss: 1311.9406 - mse: 1311.9406 - mae: 12.9951 - root_mean_squared_error: 36.2207 - mean_squared_logarithmic_error: 4.2763
Epoch 4/100
14/14 [==============================] - 0s 1ms/step - loss: 1116.5962 - mse: 1116.5962 - mae: 11.2323 - root_mean_squared_error: 33.4155 - mean_squared_logarithmic_error: 3.6993
Epoch 5/100
14/14 [==============================] - 0s 1ms/step - loss: 926.3491 - mse: 926.3491 - mae: 9.5821 - root_mean_squared_error: 30.4360 - mean_squared_logarithmic_error: 3.1200
Epoch 6/100
14/14 [===================

14/14 [==============================] - 0s 1ms/step - loss: 1.4929 - mse: 1.4929 - mae: 0.2225 - root_mean_squared_error: 1.2219 - mean_squared_logarithmic_error: 0.1082
Epoch 87/100
14/14 [==============================] - 0s 1ms/step - loss: 1.4316 - mse: 1.4316 - mae: 0.2209 - root_mean_squared_error: 1.1965 - mean_squared_logarithmic_error: 0.1078
Epoch 88/100
14/14 [==============================] - 0s 1ms/step - loss: 1.3908 - mse: 1.3908 - mae: 0.2197 - root_mean_squared_error: 1.1793 - mean_squared_logarithmic_error: 0.1074
Epoch 89/100
14/14 [==============================] - 0s 1ms/step - loss: 1.3202 - mse: 1.3202 - mae: 0.2177 - root_mean_squared_error: 1.1490 - mean_squared_logarithmic_error: 0.1070
Epoch 90/100
14/14 [==============================] - 0s 1ms/step - loss: 1.2773 - mse: 1.2773 - mae: 0.2164 - root_mean_squared_error: 1.1302 - mean_squared_logarithmic_error: 0.1066
Epoch 91/100
14/14 [==============================] - 0s 1ms/step - loss: 1.2407 - mse: 1.240

14/14 [==============================] - 0s 930us/step - loss: 4.7936 - mse: 4.7936 - mae: 0.2445 - root_mean_squared_error: 2.1894 - mean_squared_logarithmic_error: 0.1165
Epoch 72/100
14/14 [==============================] - 0s 1ms/step - loss: 4.6011 - mse: 4.6011 - mae: 0.2420 - root_mean_squared_error: 2.1450 - mean_squared_logarithmic_error: 0.1158
Epoch 73/100
14/14 [==============================] - 0s 1ms/step - loss: 4.5379 - mse: 4.5379 - mae: 0.2409 - root_mean_squared_error: 2.1302 - mean_squared_logarithmic_error: 0.1155
Epoch 74/100
14/14 [==============================] - 0s 1ms/step - loss: 4.3013 - mse: 4.3013 - mae: 0.2381 - root_mean_squared_error: 2.0740 - mean_squared_logarithmic_error: 0.1149
Epoch 75/100
14/14 [==============================] - 0s 1ms/step - loss: 4.1971 - mse: 4.1971 - mae: 0.2367 - root_mean_squared_error: 2.0487 - mean_squared_logarithmic_error: 0.1145
Epoch 76/100
14/14 [==============================] - 0s 1ms/step - loss: 4.0303 - mse: 4.0

14/14 [==============================] - 0s 1ms/step - loss: 0.3048 - mse: 0.3048 - mae: 0.1542 - root_mean_squared_error: 0.5521 - mean_squared_logarithmic_error: 0.0850
Epoch 58/100
14/14 [==============================] - 0s 1ms/step - loss: 0.3048 - mse: 0.3048 - mae: 0.1542 - root_mean_squared_error: 0.5521 - mean_squared_logarithmic_error: 0.0850
Epoch 59/100
14/14 [==============================] - 0s 1ms/step - loss: 0.3048 - mse: 0.3048 - mae: 0.1542 - root_mean_squared_error: 0.5521 - mean_squared_logarithmic_error: 0.0850
Epoch 60/100
14/14 [==============================] - 0s 1ms/step - loss: 0.3048 - mse: 0.3048 - mae: 0.1542 - root_mean_squared_error: 0.5521 - mean_squared_logarithmic_error: 0.0850
Epoch 61/100
14/14 [==============================] - 0s 1ms/step - loss: 0.3048 - mse: 0.3048 - mae: 0.1542 - root_mean_squared_error: 0.5521 - mean_squared_logarithmic_error: 0.0850
Epoch 62/100
14/14 [==============================] - 0s 1ms/step - loss: 0.3048 - mse: 0.304

14/14 [==============================] - 0s 1ms/step - loss: 1192.6206 - mse: 1192.6206 - mae: 10.7144 - root_mean_squared_error: 34.5343 - mean_squared_logarithmic_error: 3.0849
Epoch 42/100
14/14 [==============================] - 0s 1ms/step - loss: 1132.6405 - mse: 1132.6405 - mae: 10.2622 - root_mean_squared_error: 33.6547 - mean_squared_logarithmic_error: 2.9451
Epoch 43/100
14/14 [==============================] - 0s 1ms/step - loss: 1073.8009 - mse: 1073.8008 - mae: 9.8440 - root_mean_squared_error: 32.7689 - mean_squared_logarithmic_error: 2.8202
Epoch 44/100
14/14 [==============================] - 0s 1ms/step - loss: 1026.8190 - mse: 1026.8190 - mae: 9.4440 - root_mean_squared_error: 32.0440 - mean_squared_logarithmic_error: 2.6936
Epoch 45/100
14/14 [==============================] - 0s 1ms/step - loss: 971.9269 - mse: 971.9269 - mae: 9.0434 - root_mean_squared_error: 31.1757 - mean_squared_logarithmic_error: 2.5764
Epoch 46/100
14/14 [==============================] - 0s 1

...

14/14 [==============================] - 0s 4ms/step - loss: 12924.0586 - mse: 12924.0586 - mae: 47.3702 - root_mean_squared_error: 113.6840 - mean_squared_logarithmic_error: 10.6140
Epoch 2/100
14/14 [==============================] - 0s 1ms/step - loss: 12111.5879 - mse: 12111.5879 - mae: 45.4854 - root_mean_squared_error: 110.0527 - mean_squared_logarithmic_error: 10.3439
Epoch 3/100
14/14 [==============================] - 0s 967us/step - loss: 11419.0254 - mse: 11419.0254 - mae: 43.7599 - root_mean_squared_error: 106.8598 - mean_squared_logarithmic_error: 10.0846
Epoch 4/100
14/14 [==============================] - 0s 942us/step - loss: 10729.4980 - mse: 10729.4980 - mae: 42.1400 - root_mean_squared_error: 103.5833 - mean_squared_logarithmic_error: 9.8390
Epoch 5/100
14/14 [==============================] - 0s 815us/step - loss: 10080.6133 - mse: 10080.6133 - mae: 40.6056 - root_mean_squared_error: 100.4023 - mean_squared_logarithmic_error: 9.5971
Epoch 6/100
14/14 [==============

Epoch 86/100
14/14 [==============================] - 0s 860us/step - loss: 141.5602 - mse: 141.5602 - mae: 1.3051 - root_mean_squared_error: 11.8979 - mean_squared_logarithmic_error: 0.2902
Epoch 87/100
14/14 [==============================] - 0s 838us/step - loss: 135.5821 - mse: 135.5821 - mae: 1.2586 - root_mean_squared_error: 11.6440 - mean_squared_logarithmic_error: 0.2794
Epoch 88/100
14/14 [==============================] - 0s 839us/step - loss: 129.5602 - mse: 129.5602 - mae: 1.2200 - root_mean_squared_error: 11.3825 - mean_squared_logarithmic_error: 0.2732
Epoch 89/100
14/14 [==============================] - 0s 869us/step - loss: 124.0893 - mse: 124.0893 - mae: 1.1853 - root_mean_squared_error: 11.1395 - mean_squared_logarithmic_error: 0.2660
Epoch 90/100
14/14 [==============================] - 0s 863us/step - loss: 119.7360 - mse: 119.7360 - mae: 1.1573 - root_mean_squared_error: 10.9424 - mean_squared_logarithmic_error: 0.2616
Epoch 91/100
14/14 [=========================

14/14 [==============================] - 0s 851us/step - loss: 1.4865 - mse: 1.4865 - mae: 0.2367 - root_mean_squared_error: 1.2192 - mean_squared_logarithmic_error: 0.1201
Epoch 71/100
14/14 [==============================] - 0s 810us/step - loss: 1.4524 - mse: 1.4524 - mae: 0.2347 - root_mean_squared_error: 1.2052 - mean_squared_logarithmic_error: 0.1194
Epoch 72/100
14/14 [==============================] - 0s 890us/step - loss: 1.4056 - mse: 1.4056 - mae: 0.2325 - root_mean_squared_error: 1.1856 - mean_squared_logarithmic_error: 0.1186
Epoch 73/100
14/14 [==============================] - 0s 909us/step - loss: 1.3711 - mse: 1.3711 - mae: 0.2302 - root_mean_squared_error: 1.1710 - mean_squared_logarithmic_error: 0.1178
Epoch 74/100
14/14 [==============================] - 0s 825us/step - loss: 1.3335 - mse: 1.3335 - mae: 0.2283 - root_mean_squared_error: 1.1548 - mean_squared_logarithmic_error: 0.1171
Epoch 75/100
14/14 [==============================] - 0s 860us/step - loss: 1.2942 

14/14 [==============================] - 0s 814us/step - loss: 0.4575 - mse: 0.4575 - mae: 0.1744 - root_mean_squared_error: 0.6764 - mean_squared_logarithmic_error: 0.0934
Epoch 55/100
14/14 [==============================] - 0s 851us/step - loss: 0.4433 - mse: 0.4433 - mae: 0.1737 - root_mean_squared_error: 0.6658 - mean_squared_logarithmic_error: 0.0932
Epoch 56/100
14/14 [==============================] - 0s 849us/step - loss: 0.4258 - mse: 0.4258 - mae: 0.1727 - root_mean_squared_error: 0.6526 - mean_squared_logarithmic_error: 0.0929
Epoch 57/100
14/14 [==============================] - 0s 860us/step - loss: 0.4156 - mse: 0.4156 - mae: 0.1721 - root_mean_squared_error: 0.6447 - mean_squared_logarithmic_error: 0.0928
Epoch 58/100
14/14 [==============================] - 0s 846us/step - loss: 0.4080 - mse: 0.4080 - mae: 0.1716 - root_mean_squared_error: 0.6387 - mean_squared_logarithmic_error: 0.0926
Epoch 59/100
14/14 [==============================] - 0s 862us/step - loss: 0.3958 

14/14 [==============================] - 0s 915us/step - loss: 0.3050 - mse: 0.3050 - mae: 0.1553 - root_mean_squared_error: 0.5523 - mean_squared_logarithmic_error: 0.0852
Epoch 40/100
14/14 [==============================] - 0s 842us/step - loss: 0.3050 - mse: 0.3050 - mae: 0.1552 - root_mean_squared_error: 0.5523 - mean_squared_logarithmic_error: 0.0851
Epoch 41/100
14/14 [==============================] - 0s 922us/step - loss: 0.3050 - mse: 0.3050 - mae: 0.1552 - root_mean_squared_error: 0.5523 - mean_squared_logarithmic_error: 0.0851
Epoch 42/100
14/14 [==============================] - 0s 825us/step - loss: 0.3050 - mse: 0.3050 - mae: 0.1552 - root_mean_squared_error: 0.5523 - mean_squared_logarithmic_error: 0.0851
Epoch 43/100
14/14 [==============================] - 0s 844us/step - loss: 0.3050 - mse: 0.3050 - mae: 0.1551 - root_mean_squared_error: 0.5522 - mean_squared_logarithmic_error: 0.0851
Epoch 44/100
14/14 [==============================] - 0s 831us/step - loss: 0.3050 

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv285/'

In [23]:
total_training_time

[2.7366607189178467,
 2.5629971027374268,
 2.6121602058410645,
 2.6089024543762207,
 2.5956318378448486,
 2.680058240890503,
 2.6938328742980957,
 2.557100772857666,
 2.5981369018554688,
 2.6043245792388916,
 2.548841714859009,
 2.762479066848755,
 2.2966644763946533,
 2.273777484893799,
 2.281341075897217,
 2.3683793544769287,
 2.2797799110412598,
 2.1053478717803955,
 1.9927024841308594,
 1.9521918296813965,
 2.2963790893554688,
 2.1587371826171875,
 2.120286226272583,
 2.0936055183410645,
 2.1308178901672363,
 2.0662198066711426,
 2.004087448120117,
 1.9774501323699951,
 1.9523630142211914,
 2.102614164352417]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 2.3337957143783568


In [25]:
total_training_loss_history

[{'loss': [1894.5533447265625,
   1602.2652587890625,
   1311.9405517578125,
   1116.59619140625,
   926.3490600585938,
   771.5269165039062,
   652.314453125,
   540.185546875,
   458.6308898925781,
   391.38238525390625,
   331.4310607910156,
   283.18756103515625,
   248.7179718017578,
   209.6575164794922,
   182.94398498535156,
   158.3896484375,
   139.5785369873047,
   118.28936767578125,
   104.63468933105469,
   91.1962661743164,
   80.96827697753906,
   68.95790100097656,
   60.281822204589844,
   52.7804069519043,
   48.481014251708984,
   41.265689849853516,
   37.6135139465332,
   33.102901458740234,
   29.80738639831543,
   26.466270446777344,
   23.63352394104004,
   21.60812759399414,
   19.332279205322266,
   17.6937313079834,
   15.879768371582031,
   14.406384468078613,
   13.46646785736084,
   12.0719633102417,
   11.104103088378906,
   10.342276573181152,
   9.569098472595215,
   8.93749713897705,
   8.449760437011719,
   7.857780933380127,
   7.511490821838379,
  

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 6.474394027392069
avg mae of training: 0.23651776760816573


In [27]:
total_scores

[[0.24202010035514832,
  0.24202010035514832,
  0.13044069707393646,
  0.49195536971092224,
  0.07062437385320663],
 [0.3409712612628937,
  0.3409712612628937,
  0.14419251680374146,
  0.5839274525642395,
  0.07464571297168732],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [22.822946548461914,
  22.822946548461914,
  0.7708810567855835,
  4.777336597442627,
  0.2610931992530823],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.3158884644508362,
  0.3158884644508362,
  0.14192107319831848,
  0.5620395541191101,
  0.0738411620259285],
 [1.3771620988845825,
  1.3771620988845825,
  0.18890920281410217,
  1.1735254526138306,
  0.0975

In [1]:
a = [[0.24202010035514832,
  0.24202010035514832,
  0.13044069707393646,
  0.49195536971092224,
  0.07062437385320663],
 [0.3409712612628937,
  0.3409712612628937,
  0.14419251680374146,
  0.5839274525642395,
  0.07464571297168732],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [22.822946548461914,
  22.822946548461914,
  0.7708810567855835,
  4.777336597442627,
  0.2610931992530823],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.3158884644508362,
  0.3158884644508362,
  0.14192107319831848,
  0.5620395541191101,
  0.0738411620259285],
 [1.3771620988845825,
  1.3771620988845825,
  0.18890920281410217,
  1.1735254526138306,
  0.09757084399461746],
 [0.9150650501251221,
  0.9150650501251221,
  0.17641928791999817,
  0.9565903544425964,
  0.09414449334144592],
 [0.33668407797813416,
  0.33668407797813416,
  0.14382413029670715,
  0.5802448391914368,
  0.07451724261045456],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.35807281732559204,
  0.35807281732559204,
  0.14559011161327362,
  0.5983918309211731,
  0.07512807846069336],
 [0.2457883656024933,
  0.2457883656024933,
  0.13900960981845856,
  0.49577048420906067,
  0.07315953820943832],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.24191981554031372,
  0.24191981554031372,
  0.1301002949476242,
  0.4918534457683563,
  0.07054442912340164],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.4624468982219696,
  0.4624468982219696,
  0.16464920341968536,
  0.6800344586372375,
  0.08420320600271225],
 [0.3217991292476654,
  0.3217991292476654,
  0.18567721545696259,
  0.5672734379768372,
  0.09551088511943817],
 [0.27521583437919617,
  0.27521583437919617,
  0.14077343046665192,
  0.524610161781311,
  0.07718013972043991],
 [0.24377556145191193,
  0.24377556145191193,
  0.13390715420246124,
  0.4937363266944885,
  0.07175002247095108],
 [0.4802285134792328,
  0.4802285134792328,
  0.1589922457933426,
  0.6929852366447449,
  0.08731801807880402],
 [0.2947196364402771,
  0.2947196364402771,
  0.14227734506130219,
  0.5428808927536011,
  0.07859329134225845],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [0.2418879121541977,
  0.2418879121541977,
  0.12979350984096527,
  0.4918210208415985,
  0.07051555812358856],
 [2.9343626499176025,
  2.9343626499176025,
  0.2592190206050873,
  1.7129981517791748,
  0.10447091609239578],
 [0.2864116430282593,
  0.2864116430282593,
  0.1461656391620636,
  0.5351744294166565,
  0.07645120471715927],
 [0.2418903410434723,
  0.2418903410434723,
  0.1298781931400299,
  0.4918234944343567,
  0.07051791995763779],
 [0.24190790951251984,
  0.24190790951251984,
  0.13003647327423096,
  0.4918413460254669,
  0.07053404301404953]]

In [2]:
test_mae = []
for i in range(30):
    test_mae.append(a[i][2])

In [3]:
test_mae

[0.13044069707393646,
 0.14419251680374146,
 0.12979350984096527,
 0.7708810567855835,
 0.12979350984096527,
 0.12979350984096527,
 0.12979350984096527,
 0.14192107319831848,
 0.18890920281410217,
 0.17641928791999817,
 0.14382413029670715,
 0.12979350984096527,
 0.14559011161327362,
 0.13900960981845856,
 0.12979350984096527,
 0.1301002949476242,
 0.12979350984096527,
 0.16464920341968536,
 0.18567721545696259,
 0.14077343046665192,
 0.13390715420246124,
 0.1589922457933426,
 0.14227734506130219,
 0.12979350984096527,
 0.12979350984096527,
 0.12979350984096527,
 0.2592190206050873,
 0.1461656391620636,
 0.1298781931400299,
 0.13003647327423096]

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 1.1799385279417038
avg mae: 0.1666933000087738
avg are: 0.10590953752398491


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.34530195593833923
median testing mse: 0.2447819635272026


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.1655542552471161
median testing mae: 0.1364583820104599


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.09439527988433838


In [35]:
test_ARE

[0.09439528,
 0.10619469,
 0.09439528,
 0.2879056,
 0.09439528,
 0.09439528,
 0.09439528,
 0.10619469,
 0.09439528,
 0.09439528,
 0.10619469,
 0.09439528,
 0.109144546,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.112094395,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.17256637,
 0.100294985,
 0.09439528,
 0.09439528]

In [6]:
sum([0.09439528,
 0.10619469,
 0.09439528,
 0.2879056,
 0.09439528,
 0.09439528,
 0.09439528,
 0.10619469,
 0.09439528,
 0.09439528,
 0.10619469,
 0.09439528,
 0.109144546,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.112094395,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.09439528,
 0.17256637,
 0.100294985,
 0.09439528,
 0.09439528][:10])/10

0.116106194